# Learn Pyhidra

Pyhidra is a Python library that provides direct access to the Ghidra API within a native CPython interpreter using [jpype](https://jpype.readthedocs.io/en/latest). As well, Pyhidra contains some conveniences for setting up analysis on a given sample and running a Ghidra script locally. It also contains a Ghidra plugin to allow the use of CPython from the Ghidra user interface.

In this script, we will learn how to use basic functions provided by pyhidra.
Furthermore, we will discover the function related to binary analysis

* **Task 1:** wirte a inherit launcher class to implement log service
* **Task 2:** Create or open a project with given path and name 
* **Task 3:** import a program with path and name and get the flat api
    * flat_api is a interface to export many useful functions without determine the class of it
* **Task 4:** set the correct base address of the binary file
* **Task 5:** create a function based on the vector table 
* **Task 6:** Save the program and close the project


In [12]:
# remember to set the GHIDRA_INSTALL_DIR first
! pip install pyhidra > /dev/null
# import launcher
from pyhidra.launcher import PyhidraLauncher, GHIDRA_INSTALL_DIR

In [3]:
# Task 1: wirte a inherit launcher class to implement log service
# inherit PyhidraLauncher to save the log to file 
class HeadlessLoggingPyhidraLauncher(PyhidraLauncher):
    """
    Headless pyhidra launcher
    Slightly Modified from Pyhidra to allow the Ghidra log path to be set
    """

    def __init__(self, verbose=False, log_path=None):
        super().__init__(verbose)
        self.log_path = log_path

    def _launch(self):
        from pyhidra.launcher import _silence_java_output
        from ghidra.framework import Application, HeadlessGhidraApplicationConfiguration
        from java.io import File
        with _silence_java_output(not self.verbose, not self.verbose):
            config = HeadlessGhidraApplicationConfiguration()
            if self.log_path:
                log = File(self.log_path)
                config.setApplicationLogFile(log)
            Application.initializeApplication(self.layout, config)

In [4]:
# start the Launcher 
launcher = HeadlessLoggingPyhidraLauncher(verbose=True, log_path='./launch.log')
launcher.start()

INFO  Using log config file: jar:file:/data/Downloads/ghidra_11.0_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: ./launch.log (LoggingInitialization)  
INFO  Loading user preferences: /home/dingisoul/.ghidra/.ghidra_11.0_PUBLIC/preferences (Preferences)  
INFO  Searching for classes... (ClassSearcher)  
INFO  Class search complete (481 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  


In [5]:
# Task 2: Create or open a project with given path and name 
# Necessary imports for ghidra project 
from ghidra.base.project import GhidraProject
from java.io import IOException
from pathlib import Path 

# Create Project Dir and name 
project_location = Path('./ghidra_project')
project_location.mkdir(exist_ok=True, parents=True)
project_name = "test_project"

# create or open project 
try:
    project = GhidraProject.openProject(project_location, project_name, True)
    print(f'Opened project: {project.project.name}')
except IOException:
    project = GhidraProject.createProject(project_location, project_name, False)
    print(f'Created project: {project.project.name}')

INFO  Opening project: /data/dev/FirmFlaw/ghidra_project/test_project (DefaultProject)  
Opened project: test_project


In [6]:
# Task 3: import a program with path and name and get the flat api
program_name = "STM32L1xxSmartGlass-V1.21-181016.bin"
program_path = Path("./" + program_name)

# helper funcs 
def _get_language(id: str) -> "Language":
    from ghidra.program.util import DefaultLanguageService
    from ghidra.program.model.lang import LanguageID, LanguageNotFoundException
    try:
        service: "LanguageService" = DefaultLanguageService.getLanguageService()
        return service.getLanguage(LanguageID(id))
    except LanguageNotFoundException:
        # suppress the java exception
        pass
    raise ValueError("Invalid Language ID: "+id)
    
def _get_compiler_spec(lang: "Language", id: str = None) -> "CompilerSpec":
    if id is None:
        return lang.getDefaultCompilerSpec()
    from ghidra.program.model.lang import CompilerSpecID, CompilerSpecNotFoundException
    try:
        return lang.getCompilerSpecByID(CompilerSpecID(id))
    except CompilerSpecNotFoundException:
        # suppress the java exception
        pass
    lang_id = lang.getLanguageID()
    raise ValueError(f"Invalid CompilerSpecID: {id} for Language: {lang_id.toString()}")
    
# import program  
lang =  _get_language("ARM:LE:32:Cortex")
program = project.importProgram(program_path,lang , _get_compiler_spec(lang))

# get the flat api 
from ghidra.program.flatapi import FlatProgramAPI
flat_api = FlatProgramAPI(program)

INFO  Using Loader: Raw Binary (AutoImporter)  
INFO  Using Language/Compiler: ARM:LE:32:Cortex:default (AutoImporter)  
Original Image Base: 00000000
0x20001890


In [ ]:
# Task 4: set the correct base address of the binary file
# set base address
print(f'Original Image Base: {program.getImageBase()}')
base = program.getImageBase()
# setImageBase (Address base, boolean commit)
program.setImageBase(base.getNewAddress(0x8003000), True)
# test for the result
print(hex(flat_api.getInt(flat_api.toAddr(0x8003000))))

In [7]:
# Task 5: create a function based on the vector table 
# raw program has no function 
list_ = program.getListing()
funcs = list_.getFunctions(False)
print(f'has func :{funcs.hasNext()}')
# create one function 
entry_point = flat_api.toAddr(flat_api.getInt(flat_api.toAddr(0x8003004)) - 1)
print(f'Entry Point :{entry_point}')
flat_api.createFunction(entry_point, 'Reset_Handler')

has func :False
Entry Point :08003294


Reset_Handler

In [8]:
# Task 6: Save the program and close the project
project.saveAs(program, "/", program.getName(), True)
project.save(program)
project.close()

INFO  /STM32L1xxSmartGlass-V1.21-181016.bin: file deleted (dingisoul) (LocalFileSystem)  
INFO  Deleted local file STM32L1xxSmartGlass-V1.21-181016.bin (GhidraFileData)  
INFO  /udf_7f0118d9336613937455183: file deleted (LocalFileSystem)  
